In [2]:
import pandas as pd
import pickle
import json
import os
import pyarrow as pa
import fastparquet as fp

business.pkl to parquet

In [3]:
# Ruta a la carpeta Yelp
yelp_folder = 'Yelp'

file_to_df = 'business.pkl'
# Leer el archivo business.pkl
business_df = pd.read_pickle(os.path.join(yelp_folder, file_to_df))

In [6]:
business_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150346 entries, 0 to 150345
Data columns (total 28 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   address       150346 non-null  object
 3   city          150346 non-null  object
 4   state         150343 non-null  object
 5   postal_code   150346 non-null  object
 6   latitude      150346 non-null  object
 7   longitude     150346 non-null  object
 8   stars         150346 non-null  object
 9   review_count  150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150243 non-null  object
 13  hours         127123 non-null  object
 14  business_id   5 non-null       object
 15  name          5 non-null       object
 16  address       5 non-null       object
 17  city          5 non-null       object
 18  state         5 non-null     

In [7]:
# Crear un nuevo DataFrame con las primeras 14 columnas
df_business = business_df.iloc[:, :14]

In [8]:
# Filtrar las filas sin valores NA / NaN en la columna 'categories'
df_categories = df_business.dropna(subset=['categories'])

# Verificar la información del nuevo DataFrame
df_categories.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150243 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150243 non-null  object
 1   name          150243 non-null  object
 2   address       150243 non-null  object
 3   city          150243 non-null  object
 4   state         150240 non-null  object
 5   postal_code   150243 non-null  object
 6   latitude      150243 non-null  object
 7   longitude     150243 non-null  object
 8   stars         150243 non-null  object
 9   review_count  150243 non-null  object
 10  is_open       150243 non-null  object
 11  attributes    136601 non-null  object
 12  categories    150243 non-null  object
 13  hours         127123 non-null  object
dtypes: object(14)
memory usage: 17.2+ MB


In [9]:
# lista de palabras clave relacionadas con comida
palabras_clave = ["Restaurant", "restaurant", "Food", "food", "Cafe", "cafe", "Diner", "diner", "Bakery", "bakery", "Lunch", "lunch", "Brunch", 'brunch']

# Filtrar los registros que contienen al menos una de las palabras clave en la columna 'category'
df_resto = df_categories[df_categories['categories'].str.contains('|'.join(palabras_clave))]

In [10]:
# Verificar si el directorio 'parquet' existe, y si no, crearlo
if not os.path.exists('parquet'):
    os.makedirs('parquet')

# Obtener el nombre del archivo original sin la extensión
df_2_file_name = 'business.pkl'.split('.')[0]

# Guardar el DataFrame en formato Parquet en el directorio 'parquet' con el nombre del archivo original
file_path = os.path.join('parquet', f'{df_2_file_name}.parquet')

# A continuación, se guarda el DataFrame en formato Parquet usando fastparquet
df_resto.to_parquet(file_path, engine='fastparquet')

print(f"DataFrame guardado como {file_path}")

DataFrame guardado como parquet\business.parquet


In [11]:
df_sorted = df_resto.sort_values(by='business_id')
primeros_10_business_id_ordenados = df_sorted['business_id'].head(10)
print(primeros_10_business_id_ordenados)

29889     ---kPU91CF4Lq2-WlRu9Lw
118535    --0iUa4sNDFiZFrAdIWhZQ
45464     --7PUidqRWpRSpXebiyxTg
59459     --8IbOsAAxjKRoYsBFL-PA
75117     --MbOh2O1pATkXa7xbU6LA
38991     --SJXpAa0E-GCp2smaHf0A
9257      --ZVrH2X2QXBFdCilbirsw
101542    --a_r_w1HTsOY-fagPeNKg
141306    --epgcb7xHGuJ-4PUeSLAw
8466      --hF_3v1JmU9nlu4zfXJ8Q
Name: business_id, dtype: object


In [12]:
df_resto.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64629 entries, 3 to 150340
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   business_id   64629 non-null  object
 1   name          64629 non-null  object
 2   address       64629 non-null  object
 3   city          64629 non-null  object
 4   state         64629 non-null  object
 5   postal_code   64629 non-null  object
 6   latitude      64629 non-null  object
 7   longitude     64629 non-null  object
 8   stars         64629 non-null  object
 9   review_count  64629 non-null  object
 10  is_open       64629 non-null  object
 11  attributes    63752 non-null  object
 12  categories    64629 non-null  object
 13  hours         55293 non-null  object
dtypes: object(14)
memory usage: 7.4+ MB


In [13]:
# Calcular la cantidad de valores únicos en la columna 'business_id'
unique_business_ids = df_resto['business_id'].nunique()

# Imprimir la cantidad de valores únicos
print(f"Cantidad de valores únicos en df_resto['business_id']: {unique_business_ids}")

Cantidad de valores únicos en df_resto['business_id']: 64629


checkin.json to parquet

In [28]:
# Ruta al archivo checkin.json dentro de la carpeta Yelp
checkin_path = os.path.join('Yelp', 'checkin.json')

# Lista para almacenar los DataFrames
checkin_dfs = []

# Leer el archivo línea por línea y convertir cada línea en un DataFrame
with open(checkin_path, 'r') as checkin_file:
    for line in checkin_file:
        data = eval(line)  # Convertir la línea en un diccionario
        df = pd.DataFrame([data])  # Crear un DataFrame a partir del diccionario
        checkin_dfs.append(df)

# Concatenar los DataFrames en uno solo
checkin_df = pd.concat(checkin_dfs, ignore_index=True)

In [29]:
checkin_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131930 entries, 0 to 131929
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   business_id  131930 non-null  object
 1   date         131930 non-null  object
dtypes: object(2)
memory usage: 2.0+ MB


In [31]:
for column in checkin_df.columns:
    print(f"Columna: {column}")
    print(checkin_df[column].head(10))
    print("\n")

Columna: business_id
0    ---kPU91CF4Lq2-WlRu9Lw
1    --0iUa4sNDFiZFrAdIWhZQ
2    --30_8IhuyMHbSOcNWd6DQ
3    --7PUidqRWpRSpXebiyxTg
4    --7jw19RH9JKXgFohspgQw
5    --8IbOsAAxjKRoYsBFL-PA
6    --9osgUCSDUWUkoTLdvYhQ
7    --ARBQr1WMsTWiwOKOj-FQ
8    --FWWsIwxRwuw9vIMImcQg
9    --FcbSxK1AoEtEAxOgBaCw
Name: business_id, dtype: object


Columna: date
0    2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020...
1    2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011...
2             2013-06-14 23:29:17, 2014-08-13 23:20:22
3    2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012...
4    2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014...
5    2015-06-06 01:03:19, 2015-07-29 16:50:58, 2015...
6    2015-06-13 02:00:57, 2015-07-04 00:44:09, 2015...
7    2014-12-12 00:44:23, 2015-01-09 00:19:52, 2015...
8    2010-09-11 16:28:39, 2010-12-22 21:14:19, 2011...
9    2017-08-18 19:43:50, 2017-10-07 22:38:38, 2017...
Name: date, dtype: object




In [32]:
# Calcular la cantidad de valores únicos en la columna 'business_id'
unique_business_ids = checkin_df['business_id'].nunique()

# Imprimir la cantidad de valores únicos
print(f"Cantidad de valores únicos en checkin_df['business_id']: {unique_business_ids}")

Cantidad de valores únicos en 'business_id': 131930


In [49]:
# Verificar si el directorio 'parquet' existe, y si no, crearlo
if not os.path.exists('parquet'):
    os.makedirs('parquet')

# Obtener el nombre del archivo original sin la extensión
df_2_file_name = 'checkin.json'.split('.')[0]

# Guardar el DataFrame en formato Parquet en el directorio 'parquet' con el nombre del archivo original
file_path = os.path.join('parquet', f'{df_2_file_name}.parquet')

# A continuación, se guarda el DataFrame en formato Parquet usando fastparquet
checkin_df.to_parquet(file_path, engine='fastparquet')

print(f"DataFrame guardado como {file_path}")

DataFrame guardado como parquet\checkin.parquet


tip.json to parquet

In [22]:
# Ruta al archivo tip.json dentro de la carpeta Yelp
tip_path = os.path.join('Yelp', 'tip.json')

# Lista para almacenar los DataFrames
tip_df = []

# Leer el archivo línea por línea y convertir cada línea en un DataFrame
with open(tip_path, 'r', encoding='utf-8') as tip_file:
    for line in tip_file:
        data = json.loads(line)  # Convertir la línea en un diccionario
        df = pd.DataFrame([data])  # Crear un DataFrame a partir del diccionario
        tip_dfs.append(df)

# Concatenar los DataFrames en uno solo
tip_df = pd.concat(tip_df, ignore_index=True)




In [23]:
tip_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908915 entries, 0 to 908914
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_id           908915 non-null  object
 1   business_id       908915 non-null  object
 2   text              908915 non-null  object
 3   date              908915 non-null  object
 4   compliment_count  908915 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 34.7+ MB


In [14]:
for column in tip_df.columns:
    print(f"Columna: {column}")
    print(tip_df[column].head(10))
    print("\n")

Columna: user_id
0    AGNUgVwnZUey3gcPCJ76iw
1    NBN4MgHP9D3cw--SnauTkA
2    -copOvldyKh1qr-vzkDEvw
3    FjMQVZjSqY8syIO-53KFKw
4    ld0AperBXk1h6UbqmM80zw
5    trf3Qcz8qvCDKXiTgjUcEg
6    SMGAlRjyfuYu-c-22zIyOg
7    YVBB9g23nuVJ0u44zK0pSA
8    VL12EhEdT4OWqGq0nIqkzw
9    4ay-fdVks5WMerYL_htkGQ
Name: user_id, dtype: object


Columna: business_id
0    3uLgwr0qeCNMjKenHJwPGQ
1    QoezRbYQncpRqyrLH6Iqjg
2    MYoRNLb5chwjQe3c_k37Gg
3    hV-bABTK-glh5wj31ps_Jw
4    _uN0OudeJ3Zl_tf6nxg5ww
5    7Rm9Ba50bw23KTA8RedZYg
6    kH-0iXqkL7b8UXNpguBMKg
7    jtri188kuhe_AuEOJ51U_A
8    xODBZmX4EmlVvbqtKN7YKg
9    pICJRcyqW1cF96Q3XhLSbw
Name: business_id, dtype: object


Columna: text
0                       Avengers time with the ladies.
1    They have lots of good deserts and tasty cuban...
2               It's open even when you think it isn't
3                            Very decent fried chicken
4               Appetizers.. platter special for lunch
5    Chili Cup + Single Cheeseburger with onion

In [24]:
df_sorted = tip_df.sort_values(by='business_id')
primeros_10_business_id_ordenados = df_sorted['business_id'].head(10)
print(primeros_10_business_id_ordenados)

697114    ---kPU91CF4Lq2-WlRu9Lw
902156    ---kPU91CF4Lq2-WlRu9Lw
828156    ---kPU91CF4Lq2-WlRu9Lw
828684    ---kPU91CF4Lq2-WlRu9Lw
347542    --0iUa4sNDFiZFrAdIWhZQ
1106      --0iUa4sNDFiZFrAdIWhZQ
59773     --0iUa4sNDFiZFrAdIWhZQ
306601    --0iUa4sNDFiZFrAdIWhZQ
237672    --0iUa4sNDFiZFrAdIWhZQ
53982     --0iUa4sNDFiZFrAdIWhZQ
Name: business_id, dtype: object


In [33]:
# Calcular la cantidad de valores únicos en la columna 'business_id'
unique_business_ids = tip_df['business_id'].nunique()

# Imprimir la cantidad de valores únicos
print(f"Cantidad de valores únicos en 'business_id': {unique_business_ids}")

Cantidad de valores únicos en 'business_id': 106193


In [50]:
# Verificar si el directorio 'parquet' existe, y si no, crearlo
if not os.path.exists('parquet'):
    os.makedirs('parquet')

# Obtener el nombre del archivo original sin la extensión
df_2_file_name = 'tip.json'.split('.')[0]

# Guardar el DataFrame en formato Parquet en el directorio 'parquet' con el nombre del archivo original
file_path = os.path.join('parquet', f'{df_2_file_name}.parquet')

# A continuación, se guarda el DataFrame en formato Parquet usando fastparquet
tip_df.to_parquet(file_path, engine='fastparquet')

print(f"DataFrame guardado como {file_path}")

DataFrame guardado como parquet\tip.parquet


df_yelp conecatenado

In [44]:
# Crear un conjunto de valores únicos de 'business_id' en df_resto
unique_business_ids = set(df_resto['business_id'])

# Filtrar los registros en checkin_df y tip_df donde 'business_id' coincida con unique_business_ids
filtered_checkin_df = checkin_df[checkin_df['business_id'].isin(unique_business_ids)]
filtered_tip_df = tip_df[tip_df['business_id'].isin(unique_business_ids)]

# Concatenar df_resto con los DataFrames filtrados
df_yelp = pd.concat([df_resto, filtered_checkin_df, filtered_tip_df], ignore_index=True)

# Ahora df_yelp contendrá los registros de df_resto y los registros de checkin_df y tip_df donde 'business_id' esté en df_resto



In [45]:
# Calcular la cantidad de valores únicos en la columna 'business_id'
unique_business_ids = df_yelp['business_id'].nunique()

# Imprimir la cantidad de valores únicos
print(f"Cantidad de valores únicos en df_yelp['business_id']: {unique_business_ids}")

Cantidad de valores únicos en df_yelp['business_id']: 64629


In [47]:
# Verificar si el directorio 'parquet' existe, y si no, crearlo
if not os.path.exists('parquet'):
    os.makedirs('parquet')

# Obtener el nombre del archivo sin la extensión
df_2_file_name = 'yelp'

# Guardar el DataFrame en formato Parquet en el directorio 'parquet' con el nombre del archivo original
file_path = os.path.join('parquet', f'{df_2_file_name}.parquet')

# A continuación, se guarda el DataFrame en formato Parquet usando fastparquet
df_yelp.to_parquet(file_path, engine='fastparquet')

print(f"DataFrame guardado como {file_path}")

DataFrame guardado como parquet\yelp.parquet


In [1]:
df_yelp.info()

NameError: name 'df_yelp' is not defined